<a href="https://colab.research.google.com/github/NousForFreedom/taxi_call_service_pubsub/blob/main/taxi_driver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install solace-pubsubplus

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 37.9 MB/s eta 0:00:00


In [ ]:
import os
import platform
import time
import calendar
import math

# Import Solace Python  API modules from the solace package
from solace.messaging.messaging_service import MessagingService, ReconnectionListener, ReconnectionAttemptListener, \
            ServiceInterruptionListener, RetryStrategy, ServiceEvent
from solace.messaging.resources.topic import Topic
from solace.messaging.errors.pubsubplus_client_error import PubSubPlusClientError
from solace.messaging.publisher.request_reply_message_publisher import RequestReplyMessagePublisher

# Import Solace Python  API modules
from solace.messaging.messaging_service import MessagingService, ReconnectionListener, ReconnectionAttemptListener, ServiceInterruptionListener, RetryStrategy, ServiceEvent
from solace.messaging.errors.pubsubplus_client_error import PubSubPlusClientError
from solace.messaging.publisher.direct_message_publisher import PublishFailureListener, FailedPublishEvent
from solace.messaging.resources.topic_subscription import TopicSubscription
from solace.messaging.receiver.message_receiver import MessageHandler
from solace.messaging.config.solace_properties.message_properties import APPLICATION_MESSAGE_ID
from solace.messaging.resources.topic import Topic
from solace.messaging.receiver.inbound_message import InboundMessage



In [ ]:
# Broker Config
broker_props = {
  "solace.messaging.transport.host": "tcp://mr-connection-hcnx0u899eb.messaging.solace.cloud:55555",
  "solace.messaging.service.vpn-name": "aiot",
  "solace.messaging.authentication.scheme.basic.username": "Hanbinheo",
  "solace.messaging.authentication.scheme.basic.password": "ajou",
}

In [ ]:
# Import Solace Python  API modules from the solace package
from solace.messaging.messaging_service import MessagingService, ReconnectionListener, ReconnectionAttemptListener, \
                                                ServiceInterruptionListener, RetryStrategy, ServiceEvent
from solace.messaging.resources.topic import Topic
from solace.messaging.publisher.direct_message_publisher import PublishFailureListener, FailedPublishEvent
from solace.messaging.resources.topic_subscription import TopicSubscription
from solace.messaging.receiver.message_receiver import MessageHandler, InboundMessage
from solace.messaging.errors.pubsubplus_client_error import PubSubPlusClientError

### **PickRequest 받기**

In [ ]:
ride_id = "RIDE12345"

In [ ]:
# Ensure stdout is unbuffered on Windows
if platform.uname().system == 'Windows':
    os.environ["PYTHONUNBUFFERED"] = "1"

# Constants
TOPIC_PREFIX = "solace/taxi/python"

# Inner class for connection error handling
class ServiceEventHandler(ReconnectionListener, ReconnectionAttemptListener, ServiceInterruptionListener):
    def on_reconnected(self, e: ServiceEvent):
        print("\non_reconnected")
        print(f"Error cause: {e.get_cause()}")
        print(f"Message: {e.get_message()}")

    def on_reconnecting(self, e: "ServiceEvent"):
        print("\non_reconnecting")
        print(f"Error cause: {e.get_cause()}")
        print(f"Message: {e.get_message()}")

    def on_service_interrupted(self, e: "ServiceEvent"):
        print("\non_service_interrupted")
        print(f"Error cause: {e.get_cause()}")
        print(f"Message: {e.get_message()}")

# Inner class to handle received messages
class PickUpRequestHandler(MessageHandler):
    def __init__(self):
        self.received_requests = []

    def on_message(self, message: InboundMessage):
        # Extract relevant properties from the message
        timestamp = message.get_property("Timestamp")
        start_location = message.get_property("StartLocation")
        destination = message.get_property("Destination")

        # Construct the received data dictionary
        received_data = {
            "Timestamp": timestamp,
            "StartLocation": start_location,
            "Destination": destination
        }

        # Add to the list of received requests
        self.received_requests.append(received_data)

        print(f'Received PickUpRequest: {received_data}')

    # Function to print all received requests
    def print_received_requests(self):
        print("\nAll received PickUpRequests:")
        for request in self.received_requests:
            print(request)

# Build a messaging service with a reconnection strategy
messaging_service = MessagingService.builder().from_properties(broker_props)\
                        .with_reconnection_retry_strategy(RetryStrategy.parametrized_retry(20, 3))\
                        .build()

# Blocking connect thread
messaging_service.connect()
print(f'\nMessaging Service connected? {messaging_service.is_connected}')

# Event Handling for the messaging service
service_handler = ServiceEventHandler()
messaging_service.add_reconnection_listener(service_handler)
messaging_service.add_reconnection_attempt_listener(service_handler)
messaging_service.add_service_interruption_listener(service_handler)

# Define a Topic subscription for PickUpRequest messages
subscribe_topic_name = TOPIC_PREFIX + "/direct/PickRequest/driver_destination_A/>"
subscribe_topic = [TopicSubscription.of(subscribe_topic_name)]
print(f'\nSubscribed to topic: {subscribe_topic_name}')

# Build a Direct Message Receiver with the given topics and start it
direct_receiver = messaging_service.create_direct_message_receiver_builder()\
                        .with_subscriptions(subscribe_topic)\
                        .build()
direct_receiver.start()
print(f'Direct Receiver is running? {direct_receiver.is_running()}')

# Create an instance of the handler
request_handler = PickUpRequestHandler()

print("\nWaiting to receive PickUpRequest messages...\n")
try:
    # Receive messages asynchronously and handle them using the PickUpRequestHandler
    direct_receiver.receive_async(request_handler)

    # Keep the receiver running indefinitely and print received messages periodically
    try:
        while True:
            time.sleep(1)  # Print received messages every 1 seconds
            request_handler.print_received_requests()
    except KeyboardInterrupt:
        print('\nDisconnecting Messaging Service')

except PubSubPlusClientError as exception:
    print(f'\nError receiving messages: {exception}')

finally:
    print('\nTerminating receiver')
    direct_receiver.terminate()
    print('\nDisconnecting Messaging Service')
    messaging_service.disconnect()


Messaging Service connected? True

Subscribed to topic: solace/taxi/python/direct/PickRequest/driver_destination_A/>
Direct Receiver is running? True

Waiting to receive PickUpRequest messages...

Received PickUpRequest: {'Timestamp': '1719106555', 'StartLocation': '41.40338.2.17403', 'Destination': '123.52341.76.45786'}

All received PickUpRequests:
{'Timestamp': '1719106555', 'StartLocation': '41.40338.2.17403', 'Destination': '123.52341.76.45786'}
Received PickUpRequest: {'Timestamp': '1719106556', 'StartLocation': '41.40338.2.17403', 'Destination': '123.52341.76.45786'}

All received PickUpRequests:
{'Timestamp': '1719106555', 'StartLocation': '41.40338.2.17403', 'Destination': '123.52341.76.45786'}
{'Timestamp': '1719106556', 'StartLocation': '41.40338.2.17403', 'Destination': '123.52341.76.45786'}
Received PickUpRequest: {'Timestamp': '1719106557', 'StartLocation': '41.40338.2.17403', 'Destination': '123.52341.76.45786'}

All received PickUpRequests:
{'Timestamp': '1719106555', 

In [ ]:
request_handler.received_requests[-1] # 가장 최근 저장된 데이터

{'Timestamp': '1719106575',
 'StartLocation': '41.40338.2.17403',
 'Destination': '123.52341.76.45786'}

In [ ]:
request_handler.received_requests[-1]['StartLocation']

'41.40338.2.17403'

### **PickupRequestResponse request**

In [ ]:
# Ensure stdout is unbuffered on Windows
if platform.uname().system == 'Windows':
    os.environ["PYTHONUNBUFFERED"] = "1"

# Constants
unique_name = "PickupRequestor"

# Build and connect the messaging service
messaging_service = MessagingService.builder().from_properties(broker_props) \
                                     .with_reconnection_retry_strategy(RetryStrategy.parametrized_retry(20, 3)) \
                                     .build()
messaging_service.connect()
print(f'\nMessaging Service connected? {messaging_service.is_connected}')

# Create a direct message requestor
direct_requestor: RequestReplyMessagePublisher = messaging_service.request_reply() \
                                                                .create_request_reply_message_publisher_builder() \
                                                                .build()
direct_requestor.start()
print(f'\nDirect Requestor ready? {direct_requestor.is_ready()}')

# Prepare outbound message payload
timestamp = calendar.timegm(time.gmtime())

message_body = f"Timestamp: {timestamp}, Ride-ID: {ride_id}, Result: True"  # Simulating a request to accept the ride

outbound_msg_builder = messaging_service.message_builder() \
                .with_property("application", "samples") \
                .with_property("language", "Python")

# Send the request
try:
    topic = Topic.of(f"{TOPIC_PREFIX}/direct/request/{unique_name}")
    outbound_msg = outbound_msg_builder.with_application_message_id(f'{timestamp}') \
                                       .build(message_body)

    print(f'\nPublishing to topic: {topic}')
    publish_async = direct_requestor.publish(request_message=outbound_msg, request_destination=topic, reply_timeout=3000)

    try:
        # Wait for the response
        response = publish_async.result()
        print(f'\nReceived reply (body): {response.get_payload_as_string()}')
        reply_payload = response.get_payload_as_string()

        # Process the reply
        if "True" in reply_payload:
            print("\nDriver accepted the ride! Continuing the session...")
            # Keep the connection for further communication
        else:
            print("\nDriver declined the ride. Terminating the session.")
            print('Messaging service disconnected.')
            # End the session
            direct_requestor.terminate()
            messaging_service.disconnect()
    except TimeoutError:
        print("\n No driver responded in time. Retrying...")
        # Logic to retry can be added here

except PubSubPlusClientError as e:
    print(f'\nError occurred: {e}')

finally:
    # Clean up in case of any issues
    if direct_requestor.is_running:
        direct_requestor.terminate()
    if messaging_service.is_connected:
        messaging_service.disconnect()


Messaging Service connected? True

Direct Requestor ready? True

Publishing to topic: topic : solace/taxi/python/direct/request/PickupRequestor 

Received reply (body): Result: True

Driver accepted the ride! Continuing the session...


### **PickupComplete**

In [ ]:
# Constants

MAX_PICKUP_WAIT_TIME = 600  # Maximum wait time for pickup in seconds (10 minutes)

# Inner class for handling failed publishes
class PublisherErrorHandling(PublishFailureListener):
    def on_failed_publish(self, e: "FailedPublishEvent"):
        print("on_failed_publish")

# Build and connect the messaging service
messaging_service = MessagingService.builder().from_properties(broker_props) \
                    .with_reconnection_retry_strategy(RetryStrategy.parametrized_retry(20, 3)) \
                    .build()
messaging_service.connect()
print(f'Messaging Service connected? {messaging_service.is_connected}')

# Create a direct message publisher and start it
direct_publisher = messaging_service.create_direct_message_publisher_builder().build()
direct_publisher.set_publish_failure_listener(PublisherErrorHandling())
direct_publisher.start()
print(f'Direct Publisher ready? {direct_publisher.is_ready()}')

# Sample data simulating the driver's current location and the user's start location
start_location = request_handler.received_requests[-1]['StartLocation']
current_location = start_location # (should match for pickup complete)

try:
    while True:
        if current_location == start_location:  # Check if the driver is at the user's start location
            timestamp = str(calendar.timegm(time.gmtime()))

            # Build the PickupComplete message
            pickup_complete_message = messaging_service.message_builder() \
                .with_application_message_id(f'PickupComplete_{ride_id}') \
                .with_property("Timestamp", timestamp) \
                .with_property("Ride-ID", ride_id) \
                .with_property("Location", current_location) \
                .build(f'PickupComplete - {timestamp}: Ride {ride_id} at {current_location}')

            # Define the topic to send the message to the platform
            topic = Topic.of(TOPIC_PREFIX + f'/direct/pickup_complete/{ride_id}')
            direct_publisher.publish(destination=topic, message=pickup_complete_message)

            print(f'Published PickupComplete message to {topic}')
            break  # Exit after sending the message
        else:
            print("Driver is not at the start location yet.")
            time.sleep(10)  # Check again after 10 seconds

except KeyboardInterrupt:
    print('\nTerminating Publisher')
    direct_publisher.terminate()
    print('\nDisconnecting Messaging Service')
    messaging_service.disconnect()

Messaging Service connected? True
Direct Publisher ready? True
Published PickupComplete message to topic : solace/taxi/python/direct/pickup_complete/RIDE12345 


### **DropoffComplete**

In [ ]:
# Constants

MAX_DROP_OFF_WAIT_TIME = 600  # Maximum wait time for drop-off in seconds (10 minutes)

# Inner class for handling failed publishes
class PublisherErrorHandling(PublishFailureListener):
    def on_failed_publish(self, e: "FailedPublishEvent"):
        print("on_failed_publish")

# Build and connect the messaging service
messaging_service = MessagingService.builder().from_properties(broker_props) \
                    .with_reconnection_retry_strategy(RetryStrategy.parametrized_retry(20, 3)) \
                    .build()
messaging_service.connect()
print(f'Messaging Service connected? {messaging_service.is_connected}')

# Create a direct message publisher and start it
direct_publisher = messaging_service.create_direct_message_publisher_builder().build()
direct_publisher.set_publish_failure_listener(PublisherErrorHandling())
direct_publisher.start()
print(f'Direct Publisher ready? {direct_publisher.is_ready()}')

# Sample data simulating the driver's current location and the user's destination

destination_location = request_handler.received_requests[-1]['Destination']
current_location = destination_location # (should match for pickup complete)
# (should match for drop-off complete)

try:
    while True:
        if current_location == destination_location:  # Check if the driver is at the user's destination
            timestamp = str(calendar.timegm(time.gmtime()))

            # Build the DropoffComplete message
            dropoff_complete_message = messaging_service.message_builder() \
                .with_application_message_id(f'DropoffComplete_{ride_id}') \
                .with_property("Timestamp", timestamp) \
                .with_property("Ride-ID", ride_id) \
                .with_property("Location", current_location) \
                .build(f'DropoffComplete - {timestamp}: Ride {ride_id} at {current_location}')

            # Define the topic to send the message to the platform
            topic = Topic.of(TOPIC_PREFIX + f'/direct/dropoff_complete/{ride_id}')
            direct_publisher.publish(destination=topic, message=dropoff_complete_message)

            print(f'Published DropoffComplete message to {topic}')
            break  # Exit after sending the message
        else:
            print("Driver is not at the destination location yet.")
            time.sleep(10)  # Check again after 10 seconds

except KeyboardInterrupt:
    print('\nTerminating Publisher')
    direct_publisher.terminate()
    print('\nDisconnecting Messaging Service')
    messaging_service.disconnect()

Messaging Service connected? True
Direct Publisher ready? True
Published DropoffComplete message to topic : solace/taxi/python/direct/dropoff_complete/RIDE12345 
